In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [23]:
dataset=pd.read_csv('./data/Churn_Modelling.csv')  

In [24]:
x=dataset.iloc[:,3:13].values

y=dataset.iloc[:,-1].values

In [25]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

x[:,1]=LabelEncoder().fit_transform(x[:,1])
x[:,2]=LabelEncoder().fit_transform(x[:,2])
x=OneHotEncoder(categorical_features=[1]).fit_transform(x).toarray()
x=x[:,1:]
print(x.shape)
print(y.shape)

(10000, 11)
(10000,)


In [26]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [27]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [28]:
#build the ANN
from keras.models import Sequential  
from keras.layers import Dense, Dropout 

classifier=Sequential()
#classifier.add(layer)
classifier.add(Dense(6,activation='relu',input_shape=(x_train.shape[1],)))
classifier.add(Dropout(0.1)) 
classifier.add(Dense(6, activation='relu'))
classifier.add(Dense(1,activation='sigmoid'))
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_142 (Dense)            (None, 6)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_143 (Dense)            (None, 6)                 42        
_________________________________________________________________
dense_144 (Dense)            (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [29]:
classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
classifier.fit(x_train,y_train,batch_size=100,epochs=50)

Epoch 1/50
8000/8000 [==============================] - 2s 262us/step - loss: 0.6648 - acc: 0.6760
Epoch 2/50
8000/8000 [==============================] - 0s 32us/step - loss: 0.5631 - acc: 0.7803
Epoch 3/50
8000/8000 [==============================] - 0s 35us/step - loss: 0.4909 - acc: 0.8010
Epoch 4/50
8000/8000 [==============================] - 0s 35us/step - loss: 0.4603 - acc: 0.8036
Epoch 5/50
8000/8000 [==============================] - 0s 33us/step - loss: 0.4469 - acc: 0.8063
Epoch 6/50
8000/8000 [==============================] - 0s 32us/step - loss: 0.4430 - acc: 0.8055
Epoch 7/50
8000/8000 [==============================] - 0s 33us/step - loss: 0.4405 - acc: 0.8074
Epoch 8/50
8000/8000 [==============================] - 0s 32us/step - loss: 0.4366 - acc: 0.8082
Epoch 9/50
8000/8000 [==============================] - 0s 33us/step - loss: 0.4349 - acc: 0.8084
Epoch 10/50
8000/8000 [==============================] - 0s 32us/step - loss: 0.4303 - acc: 0.8122
Epoch 11/50
8000/8

In [31]:
y_pred=classifier.predict(x_test)
y_pred=(y_pred>0.5) 
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [32]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[1554   41]
 [ 235  170]]


In [33]:
import sklearn
accuracy=sklearn.metrics.accuracy_score(y_test, y_pred)
print('accuracy is ', accuracy)

accuracy is  0.862


In [35]:
new_prediction=classifier.predict(sc.transform(np.array([[0.0,0,600,1,40,3,60000,2,1,1,50000]])))
print(new_prediction.shape)
new_prediction=(new_prediction>0.5) #if y_pred>0.5 then return True
print(new_prediction.shape)

(1, 1)
(1, 1)


In [36]:
#Evaluation of the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential  
from keras.layers import Dense, Dropout 

def build_classifier():
    classifier=Sequential() 
    classifier.add(Dense(6,activation='relu',input_shape=(x_train.shape[1],)))
    classifier.add(Dense(6, activation='relu'))
    classifier.add(Dense(1,activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

classifier=KerasClassifier(build_fn=build_classifier,batch_size=300, epochs=30)

accuracies=cross_val_score(estimator=classifier, X= x_train,y=y_train,cv= 5, n_jobs=1)

Epoch 1/30
6400/6400 [==============================] - 2s 292us/step - loss: 0.7061 - acc: 0.5036
Epoch 2/30
6400/6400 [==============================] - 0s 12us/step - loss: 0.6519 - acc: 0.6291
Epoch 3/30
6400/6400 [==============================] - 0s 12us/step - loss: 0.6153 - acc: 0.7253
Epoch 4/30
6400/6400 [==============================] - 0s 11us/step - loss: 0.5881 - acc: 0.7800
Epoch 5/30
6400/6400 [==============================] - 0s 11us/step - loss: 0.5663 - acc: 0.7934
Epoch 6/30
6400/6400 [==============================] - 0s 11us/step - loss: 0.5470 - acc: 0.7977
Epoch 7/30
6400/6400 [==============================] - 0s 11us/step - loss: 0.5305 - acc: 0.7975
Epoch 8/30
6400/6400 [==============================] - 0s 11us/step - loss: 0.5164 - acc: 0.7977
Epoch 9/30
6400/6400 [==============================] - 0s 10us/step - loss: 0.5041 - acc: 0.7977
Epoch 10/30
6400/6400 [==============================] - 0s 10us/step - loss: 0.4939 - acc: 0.7978
Epoch 11/30
6400/6

In [37]:
mean=accuracies.mean()
variance=accuracies.std()

print(mean)
print(variance)

0.8072500012814998
0.017210097681180387


## Tunning ANN with grid search

In [38]:
##Tunning ANN with grid search
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense
from keras.models import Sequential

def build_classifier(optimizer): #the (optimizer)is impt so we can try dif optimizer and feed into model
    classifier=Sequential() 
    classifier.add(Dense(6,activation='relu',input_shape=(x_train.shape[1],)))
    classifier.add(Dense(6, activation='relu'))
    classifier.add(Dense(1,activation='sigmoid'))
    classifier.compile(optimizer= optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return classifier
classifier=KerasClassifier(build_fn=build_classifier) 
parameters={'batch_size':[25,100],
            'epochs':[15,50],
            'optimizer':['adam','rmsprop']}

grid_search=GridSearchCV(estimator=classifier,param_grid=parameters,scoring='accuracy', cv=5)
grid_search= grid_search.fit(x_train,y_train)
best_parameters= grid_search.best_params_
best_accuracy= grid_search.best_score_

Epoch 1/15
6400/6400 [==============================] - 3s 453us/step - loss: 0.5779 - acc: 0.7238
Epoch 2/15
6400/6400 [==============================] - 1s 122us/step - loss: 0.4705 - acc: 0.7989
Epoch 3/15
6400/6400 [==============================] - 1s 124us/step - loss: 0.4456 - acc: 0.8000
Epoch 4/15
6400/6400 [==============================] - 1s 122us/step - loss: 0.4358 - acc: 0.8033
Epoch 5/15
6400/6400 [==============================] - 1s 122us/step - loss: 0.4296 - acc: 0.8072
Epoch 6/15
6400/6400 [==============================] - 1s 127us/step - loss: 0.4240 - acc: 0.8119
Epoch 7/15
6400/6400 [==============================] - 1s 123us/step - loss: 0.4195 - acc: 0.8189
Epoch 8/15
6400/6400 [==============================] - 1s 133us/step - loss: 0.4150 - acc: 0.8220
Epoch 9/15
6400/6400 [==============================] - 1s 133us/step - loss: 0.4109 - acc: 0.8253
Epoch 10/15
6400/6400 [==============================] - 1s 129us/step - loss: 0.4075 - acc: 0.8253
Epoch 11/

6400/6400 [==============================] - 1s 130us/step - loss: 0.3601 - acc: 0.8553
Epoch 15/50
6400/6400 [==============================] - 1s 127us/step - loss: 0.3588 - acc: 0.8552
Epoch 16/50
6400/6400 [==============================] - 1s 124us/step - loss: 0.3574 - acc: 0.8566
Epoch 17/50
6400/6400 [==============================] - 1s 124us/step - loss: 0.3561 - acc: 0.8578
Epoch 18/50
6400/6400 [==============================] - 1s 126us/step - loss: 0.3551 - acc: 0.8569
Epoch 19/50
6400/6400 [==============================] - 1s 124us/step - loss: 0.3542 - acc: 0.8583
Epoch 20/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3532 - acc: 0.8594
Epoch 21/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3525 - acc: 0.8598
Epoch 22/50
6400/6400 [==============================] - 1s 125us/step - loss: 0.3516 - acc: 0.8597
Epoch 23/50
6400/6400 [==============================] - 1s 124us/step - loss: 0.3508 - acc: 0.8611
Epoch 24/50


6400/6400 [==============================] - 1s 133us/step - loss: 0.3377 - acc: 0.8609
Epoch 29/50
6400/6400 [==============================] - 1s 126us/step - loss: 0.3371 - acc: 0.8609
Epoch 30/50
6400/6400 [==============================] - 1s 126us/step - loss: 0.3365 - acc: 0.8620
Epoch 31/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3360 - acc: 0.8625
Epoch 32/50
6400/6400 [==============================] - 1s 126us/step - loss: 0.3357 - acc: 0.8631
Epoch 33/50
6400/6400 [==============================] - 1s 129us/step - loss: 0.3357 - acc: 0.8620
Epoch 34/50
6400/6400 [==============================] - 1s 133us/step - loss: 0.3352 - acc: 0.8620
Epoch 35/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3350 - acc: 0.8630
Epoch 36/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3349 - acc: 0.8627
Epoch 37/50
6400/6400 [==============================] - 1s 127us/step - loss: 0.3344 - acc: 0.8619
Epoch 38/50


6400/6400 [==============================] - 1s 132us/step - loss: 0.3938 - acc: 0.8300
Epoch 11/50
6400/6400 [==============================] - 1s 118us/step - loss: 0.3895 - acc: 0.8313
Epoch 12/50
6400/6400 [==============================] - 1s 118us/step - loss: 0.3854 - acc: 0.8327
Epoch 13/50
6400/6400 [==============================] - 1s 118us/step - loss: 0.3811 - acc: 0.8328
Epoch 14/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3771 - acc: 0.8337
Epoch 15/50
6400/6400 [==============================] - 1s 118us/step - loss: 0.3735 - acc: 0.8334
Epoch 16/50
6400/6400 [==============================] - 1s 126us/step - loss: 0.3700 - acc: 0.8352
Epoch 17/50
6400/6400 [==============================] - 1s 122us/step - loss: 0.3672 - acc: 0.8358
Epoch 18/50
6400/6400 [==============================] - 1s 121us/step - loss: 0.3647 - acc: 0.8345
Epoch 19/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3626 - acc: 0.8367
Epoch 20/50


6400/6400 [==============================] - 1s 125us/step - loss: 0.3863 - acc: 0.8427
Epoch 25/50
6400/6400 [==============================] - 1s 123us/step - loss: 0.3788 - acc: 0.8456
Epoch 26/50
6400/6400 [==============================] - 1s 120us/step - loss: 0.3707 - acc: 0.8502
Epoch 27/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3627 - acc: 0.8517
Epoch 28/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3558 - acc: 0.8575
Epoch 29/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3516 - acc: 0.8581
Epoch 30/50
6400/6400 [==============================] - 1s 120us/step - loss: 0.3477 - acc: 0.8589
Epoch 31/50
6400/6400 [==============================] - 1s 128us/step - loss: 0.3461 - acc: 0.8600
Epoch 32/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3444 - acc: 0.8587
Epoch 33/50
6400/6400 [==============================] - 1s 119us/step - loss: 0.3431 - acc: 0.8595
Epoch 34/50


6400/6400 [==============================] - 0s 31us/step - loss: 0.4156 - acc: 0.8194
Epoch 15/15
6400/6400 [==============================] - 0s 31us/step - loss: 0.4137 - acc: 0.8198
Epoch 1/15
6400/6400 [==============================] - 3s 509us/step - loss: 0.6271 - acc: 0.6966
Epoch 2/15
6400/6400 [==============================] - 0s 32us/step - loss: 0.5405 - acc: 0.7936
Epoch 3/15
6400/6400 [==============================] - 0s 31us/step - loss: 0.4974 - acc: 0.7970
Epoch 4/15
6400/6400 [==============================] - 0s 33us/step - loss: 0.4732 - acc: 0.7978
Epoch 5/15
6400/6400 [==============================] - 0s 33us/step - loss: 0.4582 - acc: 0.7978
Epoch 6/15
6400/6400 [==============================] - 0s 34us/step - loss: 0.4477 - acc: 0.7989
Epoch 7/15
6400/6400 [==============================] - 0s 34us/step - loss: 0.4400 - acc: 0.7998
Epoch 8/15
6400/6400 [==============================] - 0s 32us/step - loss: 0.4341 - acc: 0.8005
Epoch 9/15
6400/6400 [=======

6400/6400 [==============================] - 0s 35us/step - loss: 0.4886 - acc: 0.7934
Epoch 5/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4711 - acc: 0.7961
Epoch 6/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4577 - acc: 0.7977
Epoch 7/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4478 - acc: 0.8017
Epoch 8/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4414 - acc: 0.8062
Epoch 9/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4368 - acc: 0.8073
Epoch 10/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4337 - acc: 0.8098
Epoch 11/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4311 - acc: 0.8097
Epoch 12/50
6400/6400 [==============================] - 0s 34us/step - loss: 0.4290 - acc: 0.8125
Epoch 13/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.4268 - acc: 0.8152
Epoch 14/50
6400/6400 [====

6400/6400 [==============================] - 0s 34us/step - loss: 0.3784 - acc: 0.8445
Epoch 20/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.3740 - acc: 0.8473
Epoch 21/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.3700 - acc: 0.8480
Epoch 22/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.3662 - acc: 0.8494
Epoch 23/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.3630 - acc: 0.8500
Epoch 24/50
6400/6400 [==============================] - 0s 36us/step - loss: 0.3603 - acc: 0.8505
Epoch 25/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.3581 - acc: 0.8519
Epoch 26/50
6400/6400 [==============================] - 0s 35us/step - loss: 0.3563 - acc: 0.8530
Epoch 27/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.3546 - acc: 0.8527
Epoch 28/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.3533 - acc: 0.8548
Epoch 29/50
6400/6400 

Epoch 2/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.6924 - acc: 0.6028
Epoch 3/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.6548 - acc: 0.7792
Epoch 4/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.6306 - acc: 0.7919
Epoch 5/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.6059 - acc: 0.7927
Epoch 6/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.5751 - acc: 0.7936
Epoch 7/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.5417 - acc: 0.7967
Epoch 8/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.5116 - acc: 0.7981
Epoch 9/50
6400/6400 [==============================] - 0s 34us/step - loss: 0.4880 - acc: 0.8058
Epoch 10/50
6400/6400 [==============================] - 0s 34us/step - loss: 0.4696 - acc: 0.8109
Epoch 11/50
6400/6400 [==============================] - 0s 33us/step - loss: 0.4546 - acc: 0.8145
Epoch 12/50
6400/6

8000/8000 [==============================] - 1s 140us/step - loss: 0.3496 - acc: 0.8546
Epoch 17/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3474 - acc: 0.8575
Epoch 18/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3457 - acc: 0.8574
Epoch 19/50
8000/8000 [==============================] - 1s 146us/step - loss: 0.3444 - acc: 0.8582
Epoch 20/50
8000/8000 [==============================] - 1s 139us/step - loss: 0.3434 - acc: 0.8592
Epoch 21/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3426 - acc: 0.8596
Epoch 22/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3419 - acc: 0.8581
Epoch 23/50
8000/8000 [==============================] - 1s 147us/step - loss: 0.3414 - acc: 0.8592
Epoch 24/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3406 - acc: 0.8617
Epoch 25/50
8000/8000 [==============================] - 1s 140us/step - loss: 0.3401 - acc: 0.8609
Epoch 26/50


In [39]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'epochs': 50, 'optimizer': 'adam'}
0.85875
